<a href="https://colab.research.google.com/github/Rogerio-mack/VLF/blob/main/VLF_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# installs

In [1]:
!pip install names

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# imports

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import shutil
from google.colab import drive

import yaml
import json
import datetime 
import time
import random
import names

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Generate `name_conventions.json` and `name_conventions` `dict`

In [4]:
# Narrowband Files
#-------------------------------------------------------------------------------
# Filename: XXYYMMDDHHMMSSZZZ_ACCT.mat

# XX – Station ID
# YY – Year
# MM — Month
# DD — Day
# HH — Hour
# MM — Minute
# SS — Second
# ZZZ — Transmitter Callsign
# A — Usually not used
# CC — 00 for N/S channel, 01 for E/W channel
# T — Type of data
# A is low resolution (1 Hz sampling rate) amplitude
# B is low resolution (1 Hz sampling rate) phase
# C is high resolution (50 Hz sampling rate) amplitude
# D is high resolution (50 Hz sampling rate) phase
# F is high resolution (50 Hz sampling rate) effective group delay

# XXYYMMDDHHMMSSZZZ_ACCT.mat
# 01234567890123456789012345

# Braoadband Files
#-------------------------------------------------------------------------------
# Filename: XXYYMMDDHHMMSS_ACC.mat

# XX – Station ID
# YY – Year
# MM — Month
# DD — Day
# HH — Hour
# MM — Minute
# SS — Second
# A — Sampling rate. 0 for 100 kHz sampled data (VLF), 1 for 1 MHz sampled data (LF), 2 for 25 kHz sampled data (Siple station experiment).
# CC — 00 for N/S channel, 01 for E/W channel

# XXYYMMDDHHMMSS_ACC.mat
# 0123456789012345678901 

name_conventions = {
    
'narrowband' : {    
  'Station_ID':       [0,1],
  'Year':             [2,3],
  'Month':            [4,5],
  'Day':              [6,7],
  'Hour':             [8,9],
  'Minute':           [10,11],
  'Second':           [12,13],
  'Transmitter':      [14,16],
  '_':                [17,17],
  'Usually not used': [18,18],
  'CC':               [19,20],
  # CC — 00 for N/S channel, 01 for E/W channel
  'Type_ABCDF':       [21,21],
  # A is low resolution (1 Hz sampling rate) amplitude
  # B is low resolution (1 Hz sampling rate) phase
  # C is high resolution (50 Hz sampling rate) amplitude
  # D is high resolution (50 Hz sampling rate) phase
  # F is high resolution (50 Hz sampling rate) effective group delay
  '.mat':             [22,25]
  },

'broadband' : { 
  'Station_ID':       [0,1],
  'Year':             [2,3],
  'Month':            [4,5],
  'Day':              [6,7],
  'Hour':             [8,9],
  'Minute':           [10,11],
  'Second':           [12,13],
  '_':                [14,14],
  'A':                [15,15],
  # A — Sampling rate. 0 for 100 kHz sampled data (VLF), 1 for 1 MHz sampled data (LF), 2 for 25 kHz sampled data (Siple station experiment). 
  'CC':               [16,17],
  # CC — 00 for N/S channel, 01 for E/W channel
  '.mat':             [18,21]
  }  

}

import json 

f = open("name_conventions.json", "w")
f.write(json.dumps(name_conventions))
f.close()

with open("name_conventions.json") as json_file:
  name_conventions = json.load(json_file)

print(name_conventions)
print(name_conventions['narrowband'])




{'narrowband': {'Station_ID': [0, 1], 'Year': [2, 3], 'Month': [4, 5], 'Day': [6, 7], 'Hour': [8, 9], 'Minute': [10, 11], 'Second': [12, 13], 'Transmitter': [14, 16], '_': [17, 17], 'Usually not used': [18, 18], 'CC': [19, 20], 'Type_ABCDF': [21, 21], '.mat': [22, 25]}, 'broadband': {'Station_ID': [0, 1], 'Year': [2, 3], 'Month': [4, 5], 'Day': [6, 7], 'Hour': [8, 9], 'Minute': [10, 11], 'Second': [12, 13], '_': [14, 14], 'A': [15, 15], 'CC': [16, 17], '.mat': [18, 21]}}
{'Station_ID': [0, 1], 'Year': [2, 3], 'Month': [4, 5], 'Day': [6, 7], 'Hour': [8, 9], 'Minute': [10, 11], 'Second': [12, 13], 'Transmitter': [14, 16], '_': [17, 17], 'Usually not used': [18, 18], 'CC': [19, 20], 'Type_ABCDF': [21, 21], '.mat': [22, 25]}


# `get_from_name()`

In [5]:
def get_from_name(fname, field, type_band='narrowband', name_conventions=name_conventions):
  
  return fname[ int(name_conventions[type_band][field][0]) : int(name_conventions[type_band][field][1]) + 1 ]


with open("name_conventions.json") as json_file:
  name_conventions = json.load(json_file)
      

fname = 'AL110505000000NAA_101A.mat'
print(get_from_name(fname,'Station_ID'))

for key in name_conventions['narrowband'].keys():
  print(key,' : ', get_from_name(fname,key))

AL
Station_ID  :  AL
Year  :  11
Month  :  05
Day  :  05
Hour  :  00
Minute  :  00
Second  :  00
Transmitter  :  NAA
_  :  _
Usually not used  :  1
CC  :  01
Type_ABCDF  :  A
.mat  :  .mat


# `what_type()`

In [6]:
import datetime 

def what_type(path, fname):
# narrowband 
# XXYYMMDDHHMMSSZZZ_ACCT.mat
# 01234567890123456789012345

# broadband
# XXYYMMDDHHMMSS_ACC.mat
# 0123456789012345678901 

  if fname.split('.')[-1] != 'mat': return None
  if fname.count('_') != 1: return None
  if fname.count('.') != 1: return None

  try:
    datetime.datetime(int('20' + get_from_name(fname,'Year')),
                      int(get_from_name(fname,'Month')),
                      int(get_from_name(fname,'Day')),
                      int(get_from_name(fname,'Hour')),
                      int(get_from_name(fname,'Minute')),
                      int(get_from_name(fname,'Second')))
  except:
    return None

  for type_band in ['narrowband','broadband']:
    if (fname.find('_') ==  name_conventions[type_band]['_'][0]) and (fname.find('.mat') ==  name_conventions[type_band]['.mat'][0]):
      return type_band  

  return None

print('ok')
print( what_type(None,'MH210113000000DHO_101A.mat') )
print( what_type(None,'CH200721053000_001.mat') )
print( what_type(None,'CH000721053000_001.mat') )
print()

print('nok')
print( what_type(None,'CH0a0721053000_001.mat') )
print( what_type(None,'CH000721053060_001.mat') )
print( what_type(None,'CH222021053000_001.mat') )
print( what_type(None,'MH210160000000DHO_101A.mat') )
print( what_type(None,'MH210113000060DHO_101A.mat') )
print( what_type(None,'MH210140000000DHO_101A.mat') )
print( what_type(None,'teste1.mat') )
print( what_type(None,'teste2_asd.mat') )
print( what_type(None,'teste2_asd.csv') )

ok
narrowband
broadband
broadband

nok
None
None
None
None
None
None
None
None
None


# `sample_waldo()` (generate sample)

In [7]:
for dir in ['/content/Emilia','/content/Jean-Pierre','/content/narrowband','/content/broadband']:
  shutil.rmtree(dir, ignore_errors=True)
  print("Deleted '%s' directory successfully" % dir)

Deleted '/content/Emilia' directory successfully
Deleted '/content/Jean-Pierre' directory successfully
Deleted '/content/narrowband' directory successfully
Deleted '/content/broadband' directory successfully


In [8]:
def sample_waldo(root, path_out, year, max_stations=3, max_dates=2, max_files=2, max_trash=2, seed=1):
  random.seed(seed)
  
  root_dirs = os.listdir(root) 

  if not os.path.exists(path_out):
    os.mkdir(path_out)
    print("Directory ", path_out ," was created")
  else:    
    print("Warning: directory ", path_out," already exists")

  random_station_dirs = random.sample(root_dirs,max_stations)

  for d in random_station_dirs:
    try:
      random_date_dirs = random.sample(os.listdir(os.path.join(root, d)),max_dates)  
    except: 
      random_date_dirs = os.listdir(os.path.join(root, d))

    for date_dir in random_date_dirs:
      for path, subdirs, files in os.walk(os.path.join( root, d, date_dir) ):
          for fname in files[0:max_files]:
              # print(os.path.join(path, fname))
              f = open(os.path.join(path_out, fname[0:2] + year[2:4] + fname[4::]), "w")
              # f.write('hello')
              f.close()

    for i in range(max_trash):
      f = open(os.path.join(path_out, names.get_first_name() + '.csv'), "w")
      f.close()      

  return

# narrowband
root = '/content/drive/Shareddrives/NarrowbandFull_2011'
sample_waldo(root=root, path_out='/content/Emilia', year='2020')
sample_waldo(root=root, path_out='/content/Emilia', year='2021')
sample_waldo(root=root, path_out='/content/Emilia/2022', year='2022')
sample_waldo(root=root, path_out='/content/Jean-Pierre', year='2021')
sample_waldo(root=root, path_out='/content/Jean-Pierre', year='2020')

# broadband
root = '/content/drive/Shareddrives/Broadband_2011'
sample_waldo(root=root, path_out='/content/Emilia', year='2020')
sample_waldo(root=root, path_out='/content/Emilia', year='2021')
sample_waldo(root=root, path_out='/content/Emilia/broadband', year='2022')
sample_waldo(root=root, path_out='/content/Jean-Pierre', year='2022')
sample_waldo(root=root, path_out='/content/Jean-Pierre/Raulin', year='2021')
sample_waldo(root=root, path_out='/content/Jean-Pierre/Raulin', year='2022')



Directory  /content/Emilia  was created
Directory  /content/Emilia/2022  was created
Directory  /content/Jean-Pierre  was created
Directory  /content/Emilia/broadband  was created
Directory  /content/Jean-Pierre/Raulin  was created


# `waldo_dict_stations()`

In [9]:
def waldo_dict_stations(root, dict_stations=None):

  root_dirs = os.listdir(root) 

  station_dirs = root_dirs

  max_files = 3

  if dict_stations is None:
    dict_stations = {}

  for d in station_dirs:
    date_dirs = os.listdir(os.path.join(root, d))

    for date_dir in date_dirs[0:2]:
      for path, subdirs, files in os.walk(os.path.join( root, d, date_dir) ):
          for fname in files[0:max_files]:
            if what_type(None,fname) in ['narrowband','broadband']:
              dict_stations[get_from_name(fname, 'Station_ID', type_band=what_type(None,fname))] = d
              continue

  return dict_stations

# narrowband
root = '/content/drive/Shareddrives/NarrowbandFull_2011'
dict_stations = waldo_dict_stations(root, dict_stations=None)

# broadband
dict_stations = waldo_dict_stations(root, dict_stations=dict_stations)

for Station_id, Station_name in dict_stations.items():
  print(Station_id, ' ', Station_name)


SP   SouthPole
AD   Adelaide
YK   Yakutat
WS   Walsenburg
AL   Algeria
VZ   Valdez
TK   Uzbekistan
AO   Arecibo
AR   Argentina
TA   Taylor
SI   SienaCollege
AZ   Azerbaijan
SH   Sheridan
BL   Serbia
SB   SdeBoker
PF   PokerFlat
PE   Perth
PA   PalmerStation
BE   Bermuda
OW   OklahomaWest
OS   OklahomaSouth
ON   OklahomaNorth
OE   OklahomaEast
OC   OklahomaCentral
NT   Nainital
MY   Malaysia
CH   Chistochina
LV   LasVegas
CP   Cyprus
KT   Ketchikan
JU   Juneau
EC   Ecuador
NN   France
JP   Japan
CR   Greece
DB   Ireland
MH   Hermon
KW   KeyWest


# `create_df_files()`

In [10]:
def create_df_files(root):

  import pandas as pd 

  paths = []
  fnames = []
  type_bands = []
  full_names = []

  for path, subdirs, files in os.walk(root):
      for fname in files:
          paths.append(path)
          fnames.append(fname)
          full_names.append(os.path.join(path, fname))
          type_bands.append(str(what_type(path,fname)))

  df_files = pd.DataFrame()
  df_files['path'] = paths
  df_files['fname'] = fnames
  df_files['full_name'] = full_names
  df_files['type_band'] = type_bands
  df_files['copied'] = False
  df_files['copy_date'] = np.nan
  df_files['copy_date'] = df_files['copy_date'].astype('datetime64[ns]')
  df_files['to_path'] = ' '
  
  return df_files

root = '/content/Emilia'
df_files = create_df_files(root)

display(df_files.head())
print()
print(df_files['path'].value_counts())
print()
print(df_files['type_band'].value_counts())

,path,fname,full_name,type_band,copied,copy_date,to_path
0,/content/Emilia,AL210615000000DHO_101A.mat,/content/Emilia/AL210615000000DHO_101A.mat,narrowband,False,NaT,
1,/content/Emilia,MH200131000000DHO_100A.mat,/content/Emilia/MH200131000000DHO_100A.mat,narrowband,False,NaT,
2,/content/Emilia,JP210311000000_000.mat,/content/Emilia/JP210311000000_000.mat,broadband,False,NaT,
3,/content/Emilia,MH210131000000DHO_100A.mat,/content/Emilia/MH210131000000DHO_100A.mat,narrowband,False,NaT,
4,/content/Emilia,AR200923000000DHO_101A.mat,/content/Emilia/AR200923000000DHO_101A.mat,narrowband,False,NaT,



/content/Emilia              54
/content/Emilia/2022         18
/content/Emilia/broadband    16
Name: path, dtype: int64

narrowband    36
broadband     30
None          22
Name: type_band, dtype: int64


# `class file_os`

In [11]:
class file_os:
  def __init__(self, path):
    self.path = path
    self.file_type = 'os'

  def copy_file(self,file_in,file_out):
    import shutil

    shutil.copy2(file_in,file_out)
    
    return  

# `copy_files()` e `report_summary()`

In [19]:
import shutil
import time
from tqdm.auto import tqdm

def report_summary(df_files):
  print('Total files: '.ljust(27), len(df_files))
  print('All copied: '.ljust(27), len(df_files[df_files.copied == True]))
  print('Files remained to be copy: '.ljust(27), len(df_files[ (df_files.copied == False) & (df_files.type_band != 'None')]))
  print('...narrowband: '.ljust(27), len(df_files[ (df_files.copied == False) & (df_files.type_band == 'narrowband')]))
  print('...broadband: '.ljust(27), len(df_files[ (df_files.copied == False) & (df_files.type_band == 'broadband')]))
  print('To be skipped: '.ljust(27), len(df_files[ df_files.type_band == 'None' ]))
  print('All copies...')
  print(pd.DataFrame(df_files.copy_date.value_counts()))
  return

def copy_files(df_files, root_path_out, type_path_out='os', override=False, dict_stations=None):

  pbar = tqdm(desc='Files processing', total=len(df_files))

  copy_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  count_files = 0
  count_copied = 0

  print('\nBefore executing...')
  report_summary(df_files) 

  for ix, row in df_files.iterrows():
    pbar.update(1)

    path = row['path']
    fname = row['fname']
    full_name = row['full_name']
    type_band = row['type_band']
    count_files += 1

    if type_band == 'None': continue
    
    if type_band not in ['narrowband','broadband']:
      sys.exit('There is an inconsistent type_band in df_files.')

    if row['copied'] and not override: continue

    dir_date =  '20' + get_from_name(fname,'Year') + '_' + \
                get_from_name(fname,'Month') + '_' + \
                get_from_name(fname,'Day')

    dir_station = get_from_name(fname,'Station_ID')

    if dict_stations is not None:
      if dir_station in dict_stations.keys():
        dir_station = dict_stations[dir_station]

    path_out = os.path.join(root_path_out, type_band)
    if not os.path.exists(path_out):
      os.mkdir(path_out)
      None # print("Directory ", path_out ," was created")
    else:    
      None # print("Warning: directory ", path_out," already exists")

    path_out = os.path.join(path_out, dir_station)
    if not os.path.exists(path_out):
      os.mkdir(path_out)
      None # print("Directory ", path_out ," was created")
    else:    
      None # print("Warning: directory ", path_out," already exists")

    path_out = os.path.join(path_out, dir_date)
    if not os.path.exists(path_out):
      os.mkdir(path_out)
      None # print("Directory ", path_out ," was created")
    else:    
      None # print("Warning: directory ", path_out," already exists")  

    if type_path_out == 'os': instance = file_os(path)

    try:
      instance.copy_file(os.path.join(path, fname), os.path.join(path_out, fname))
      count_copied += 1
    except:
      print('A error ocurred coping file ' + os.path.join(path, fname))

    df_files.at[ix,'copied'] = True
    df_files.at[ix,'copy_date'] = copy_date 
    df_files.at[ix,'to_path'] = path_out

    # time.sleep(1)
    
  pbar.close()

  print('\nThis running...')
  print('copy_date: ', copy_date, 'count_files: ', count_files, 'count_copied: ', count_copied)
  
  print('\nAfter executing...')
  report_summary(df_files) 

  return copy_date, count_files, count_copied

root_path_out = '/content'
copy_files(df_files, root_path_out, dict_stations=dict_stations)

for ix in np.random.randint(0, len(df_files), 30):
  df_files.at[ix,'copied'] = False

copy_files(df_files, root_path_out, dict_stations=dict_stations)

print()
display(df_files.head())
                      


Files processing:   0%|          | 0/88 [00:00<?, ?it/s]


Before executing...
Total files:                88
All copied:                 66
Files remained to be copy:  0
...narrowband:              0
...broadband:               0
To be skipped:              22
All copies...
                     copy_date
2022-10-01 18:33:06         33
2022-10-01 18:33:15         33

This running...
copy_date:  2022-10-01 18:34:00 count_files:  88 count_copied:  0

After executing...
Total files:                88
All copied:                 66
Files remained to be copy:  0
...narrowband:              0
...broadband:               0
To be skipped:              22
All copies...
                     copy_date
2022-10-01 18:33:06         33
2022-10-01 18:33:15         33


Files processing:   0%|          | 0/88 [00:00<?, ?it/s]


Before executing...
Total files:                88
All copied:                 48
Files remained to be copy:  18
...narrowband:              9
...broadband:               9
To be skipped:              22
All copies...
                     copy_date
2022-10-01 18:33:06         33
2022-10-01 18:33:15         33

This running...
copy_date:  2022-10-01 18:34:00 count_files:  88 count_copied:  18

After executing...
Total files:                88
All copied:                 66
Files remained to be copy:  0
...narrowband:              0
...broadband:               0
To be skipped:              22
All copies...
                     copy_date
2022-10-01 18:33:06         26
2022-10-01 18:33:15         22
2022-10-01 18:34:00         18



,path,fname,full_name,type_band,copied,copy_date,to_path
0,/content/Emilia,AL210615000000DHO_101A.mat,/content/Emilia/AL210615000000DHO_101A.mat,narrowband,True,2022-10-01 18:34:00,/content/narrowband/Algeria/2021_06_15
1,/content/Emilia,MH200131000000DHO_100A.mat,/content/Emilia/MH200131000000DHO_100A.mat,narrowband,True,2022-10-01 18:33:15,/content/narrowband/Hermon/2020_01_31
2,/content/Emilia,JP210311000000_000.mat,/content/Emilia/JP210311000000_000.mat,broadband,True,2022-10-01 18:34:00,/content/broadband/Japan/2021_03_11
3,/content/Emilia,MH210131000000DHO_100A.mat,/content/Emilia/MH210131000000DHO_100A.mat,narrowband,True,2022-10-01 18:33:15,/content/narrowband/Hermon/2021_01_31
4,/content/Emilia,AR200923000000DHO_101A.mat,/content/Emilia/AR200923000000DHO_101A.mat,narrowband,True,2022-10-01 18:33:15,/content/narrowband/Argentina/2020_09_23


# `get_file_properties()`

In [13]:
def get_file_properties(path_file, verbose=False):
  import os  
  import yaml

  type = 'Undentified'  
  extension = '-'  
  size = 0

  if os.path.isdir(path_file):  
    type = 'Dir'  
    extension = '-'
    for f in os.scandir(path_file):
      size = size + os.stat(f).st_size
    nr_files = len(os.listdir(path_file))
  elif os.path.isfile(path_file):  
    type = 'File'  
    extension = path_file.split('/')[-1].split('.')[-1]
    size = os.path.getsize(path_file)

  file_properties = {}
  file_properties['path_file'] = path_file
  file_properties['type'] = type
  if type == 'Dir':
    file_properties['nr_files'] = nr_files
  file_properties['extension'] = extension
  file_properties['size'] = size

  if verbose:
    print(yaml.dump(file_properties, default_flow_style=False))

  return file_properties 

_ = get_file_properties('/content/drive/MyDrive/VLF/data',verbose=True)
_ = get_file_properties('/content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10',verbose=True)
_ = get_file_properties('/content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10/FE070710000500NAA_006A.mat',verbose=True)


extension: '-'
nr_files: 2
path_file: /content/drive/MyDrive/VLF/data
size: 8192
type: Dir

extension: '-'
nr_files: 2
path_file: /content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10
size: 412575
type: Dir

extension: mat
path_file: /content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10/FE070710000500NAA_006A.mat
size: 206292
type: File



# `get_file_tree1()`

In [14]:
def get_file_tree1(path,verbose=False):
  import os  
  import yaml

  if get_file_properties(path)['type'] != 'Dir':
    print(path, ' is not a path')
    return

  file_tree1 = {}
  file_tree1['path'] = path
  file_tree1['properties'] = get_file_properties(path)

  tree = {}
  for f in os.listdir(path):
    tree[f] = get_file_properties(path + '/' + f)
    
  file_tree1['files'] = tree
  
  if verbose:
    print(yaml.dump(file_tree1, default_flow_style=False))

  return  file_tree1

all_files = get_file_tree1('/content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10',verbose=True)

files:
  FE070710000500NAA_006A.mat:
    extension: mat
    path_file: /content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10/FE070710000500NAA_006A.mat
    size: 206292
    type: File
  FE070710000500NAA_006B.mat:
    extension: mat
    path_file: /content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10/FE070710000500NAA_006B.mat
    size: 206283
    type: File
path: /content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10
properties:
  extension: '-'
  nr_files: 2
  path_file: /content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10
  size: 412575
  type: Dir



In [15]:
all_files = get_file_tree1(df_files[df_files.copied == True]['to_path'][0],verbose=True)

files:
  AL210615000000DHO_100A.mat:
    extension: mat
    path_file: /content/narrowband/Algeria/2021_06_15/AL210615000000DHO_100A.mat
    size: 0
    type: File
  AL210615000000DHO_101A.mat:
    extension: mat
    path_file: /content/narrowband/Algeria/2021_06_15/AL210615000000DHO_101A.mat
    size: 0
    type: File
path: /content/narrowband/Algeria/2021_06_15
properties:
  extension: '-'
  nr_files: 2
  path_file: /content/narrowband/Algeria/2021_06_15
  size: 0
  type: Dir



# `get_format_file_info()`

In [16]:
def get_format_file_info(pathfile,type='narrowband',verbose=False):

  import yaml

  narrowband_format_dict = {
  'Station_ID':       [0,1],
  'Year':             [2,3],
  'Month':            [4,5],
  'Day':              [6,7],
  'Hour':             [8,9],
  'Minute':           [10,11],
  'Second':           [12,13],
  'Transmitter':      [14,16],
  '_':                [17,17],
  'Usually not used': [18,18],
  'CC':               [19,20],
  # CC — 00 for N/S channel, 01 for E/W channel
  'Type_ABCDF':       [21,21],
  # A is low resolution (1 Hz sampling rate) amplitude
  # B is low resolution (1 Hz sampling rate) phase
  # C is high resolution (50 Hz sampling rate) amplitude
  # D is high resolution (50 Hz sampling rate) phase
  # F is high resolution (50 Hz sampling rate) effective group delay
  '.mat':             [22,25]
  }

  broadband_format_dict = {
  'Station_ID':       [0,1],
  'Year':             [2,3],
  'Month':            [4,5],
  'Day':              [6,7],
  'Hour':             [8,9],
  'Minute':           [10,11],
  'Second':           [12,13],
  '_':                [14,14],
  'A':                [15,15],
  # A — Sampling rate. 0 for 100 kHz sampled data (VLF), 1 for 1 MHz sampled data (LF), 2 for 25 kHz sampled data (Siple station experiment). 
  'CC':               [16,17],
  # CC — 00 for N/S channel, 01 for E/W channel
  '.mat':             [18,21]
  }  

#  CC_map = {'00':'N/S','01':'E/W'} ...I have found 06 in AntarticaFerraz

  Type_ABCDF_map = {'A': ['low','amplitude'],
  'B': ['low','phase'],
  'C': ['high','amplitude'],
  'D': ['high','phase'],
  'F': ['high','group delay']
  }

  info = {}
  
  if type == 'N':
    info['filename'] = pathfile.split('/')[-1] 
    info['location'] = pathfile
    info['size'] = get_file_properties(pathfile)['size']
    for key in narrowband_format_dict.keys():
      info[key] = info['filename'][narrowband_format_dict[key][0]:narrowband_format_dict[key][1]+1]
#    info['CC'] = CC_map[info['CC']]
    info['resolution'] = Type_ABCDF_map[info['Type_ABCDF']][0]
    info['amplitude_phase'] = Type_ABCDF_map[info['Type_ABCDF']][1]
    
    info['Station_name'] = pathfile.split('/')[-3] 

  if verbose:
    print(yaml.dump(info, default_flow_style=False))

  return info 

_ = get_format_file_info('/content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10/FE070710000500NAA_006A.mat',type='N',verbose=True)
_ = get_format_file_info('/content/narrowband/AZ/2022_04_09/AZ220409000000GBZ_101A.mat',type='N',verbose=True)


.mat: .mat
CC: '06'
Day: '10'
Hour: '00'
Minute: '05'
Month: '07'
Second: '00'
Station_ID: FE
Station_name: AntarticaFerraz
Transmitter: NAA
Type_ABCDF: A
Usually not used: '0'
Year: '07'
_: _
amplitude_phase: amplitude
filename: FE070710000500NAA_006A.mat
location: /content/drive/MyDrive/VLF/data/Narrowband/AntarticaFerraz/2007_07_10/FE070710000500NAA_006A.mat
resolution: low
size: 206292

.mat: .mat
CC: '01'
Day: 09
Hour: '00'
Minute: '00'
Month: '04'
Second: '00'
Station_ID: AZ
Station_name: AZ
Transmitter: GBZ
Type_ABCDF: A
Usually not used: '1'
Year: '22'
_: _
amplitude_phase: amplitude
filename: AZ220409000000GBZ_101A.mat
location: /content/narrowband/AZ/2022_04_09/AZ220409000000GBZ_101A.mat
resolution: low
size: 0

